# SparkR: Frontend to use Apache Spark from R.

In [1]:
version

               _                           
platform       x86_64-conda-linux-gnu      
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          4                           
minor          0.3                         
year           2020                        
month          10                          
day            10                          
svn rev        79318                       
language       R                           
version.string R version 4.0.3 (2020-10-10)
nickname       Bunny-Wunnies Freak Out     

In [2]:
library("SparkR")


Attaching package: ‘SparkR’


The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var, window


The following objects are masked from ‘package:base’:

    as.data.frame, colnames, colnames<-, drop, endsWith, intersect,
    rank, rbind, sample, startsWith, subset, summary, transform, union




In [3]:
sparkR.session()

Spark package found in SPARK_HOME: /usr/local/spark



Launching java with spark-submit command /usr/local/spark/bin/spark-submit   sparkr-shell /tmp/RtmpcZVXNC/backend_port2b0e74d16f3db 


Java ref type org.apache.spark.sql.SparkSession id 1 

# Fit a k-means model with spark.kmeans

In [4]:
t <- as.data.frame(Titanic)

In [5]:
head(t)

,Class,Sex,Age,Survived,Freq
,<fct>,<fct>,<fct>,<fct>,<dbl>
1,1st,Male,Child,No,0
2,2nd,Male,Child,No,0
3,3rd,Male,Child,No,35
4,Crew,Male,Child,No,0
5,1st,Female,Child,No,0
6,2nd,Female,Child,No,0


In [9]:
training <- createDataFrame(t)
df_list <- randomSplit(training, c(7,3), 2)
kmeansDF <- df_list[[1]]
kmeansTestDF <- df_list[[2]]
kmeansModel <- spark.kmeans(kmeansDF, ~ Class + Sex + Age + Freq, k = 3)

# Model summary

In [10]:
summary(kmeansModel)

$k
[1] 3

$coefficients
  Class_1st Class_2nd Class_3rd Sex_Female Age_Child       Freq
1 0.0000000 0.0000000 0.0000000  0.0000000 0.0000000 670.000000
2 0.2777778 0.2222222 0.2222222  0.5555556 0.7777778   8.333333
3 0.3333333 0.3333333 0.3333333  0.5000000 0.0000000  97.000000

$size
$size[[1]]
[1] 1

$size[[2]]
[1] 18

$size[[3]]
[1] 6


$cluster
SparkDataFrame[prediction:int]

$is.loaded
[1] FALSE

$clusterSize
[1] 3


# Get fitted result from the k-means model

In [11]:
head(fitted(kmeansModel))

,Class,Sex,Age,Survived,Freq,prediction
,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>
1,1st,Female,Adult,No,4,1
2,1st,Female,Adult,Yes,140,2
3,1st,Female,Child,No,0,1
4,1st,Female,Child,Yes,1,1
5,1st,Male,Adult,Yes,57,2
6,1st,Male,Child,No,0,1


# Prediction

In [12]:
kmeansPredictions <- predict(kmeansModel, kmeansTestDF)
head(kmeansPredictions)

,Class,Sex,Age,Survived,Freq,prediction
,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>
1,1st,Male,Adult,No,118,2
2,2nd,Female,Adult,No,13,1
3,2nd,Male,Child,No,0,1
4,3rd,Female,Adult,No,89,2
5,3rd,Male,Adult,No,387,0
6,Crew,Female,Child,No,0,1


In [13]:
t

Class,Sex,Age,Survived,Freq
<fct>,<fct>,<fct>,<fct>,<dbl>
1st,Male,Child,No,0
2nd,Male,Child,No,0
3rd,Male,Child,No,35
Crew,Male,Child,No,0
1st,Female,Child,No,0
2nd,Female,Child,No,0
3rd,Female,Child,No,17
Crew,Female,Child,No,0
1st,Male,Adult,No,118


In [17]:
## Initialize SparkSession
sparkR.session(appName = "SparkR-data-manipulation-example")

Java ref type org.apache.spark.sql.SparkSession id 1 

In [26]:
flightsCsvPath <- "nycflights.csv"

In [27]:
# Create a local R dataframe
flights_df <- read.csv(flightsCsvPath, header = TRUE)
#flights_df$date <- as.Date(flights_df$date)

In [28]:
flights_df

year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>
2013,6,30,940,15,1216,-4,VX,N626VA,407,JFK,LAX,313,2475,9,40
2013,5,7,1657,-3,2104,10,DL,N3760C,329,JFK,SJU,216,1598,16,57
2013,12,8,859,-1,1238,11,DL,N712TW,422,JFK,LAX,376,2475,8,59
2013,5,14,1841,-4,2122,-34,DL,N914DL,2391,JFK,TPA,135,1005,18,41
2013,7,21,1102,-3,1230,-8,9E,N823AY,3652,LGA,ORF,50,296,11,2
2013,1,1,1817,-3,2008,3,AA,N3AXAA,353,LGA,ORD,138,733,18,17
2013,12,9,1259,14,1617,22,WN,N218WN,1428,EWR,HOU,240,1411,12,59
2013,8,13,1920,85,2032,71,B6,N284JB,1407,JFK,IAD,48,228,19,20
2013,9,26,725,-10,1027,-8,AA,N3FSAA,2279,LGA,MIA,148,1096,7,25


In [29]:
## Filter flights whose destination is San Francisco and write to a local data frame
SFO_df <- flights_df[flights_df$dest == "SFO", ]

In [30]:
SFO_df

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>
20,2013,8,18,1638,8,1942,-17,VX,N849VA,27,JFK,SFO,334,2586,16,38
117,2013,12,30,2202,292,130,283,B6,N595JB,415,JFK,SFO,361,2586,22,2
137,2013,1,4,1942,-3,2249,-40,B6,N637JB,645,JFK,SFO,333,2586,19,42
198,2013,10,10,1155,-5,1459,-16,UA,N512UA,766,JFK,SFO,345,2586,11,55
290,2013,10,6,1832,-7,2202,6,UA,N554UA,389,JFK,SFO,333,2586,18,32
295,2013,10,14,1725,-4,2040,-25,VX,N631VA,193,EWR,SFO,345,2565,17,25
300,2013,3,19,816,16,1146,21,UA,N525UA,397,JFK,SFO,357,2586,8,16
310,2013,1,20,1822,-8,2222,20,UA,N525UA,272,JFK,SFO,371,2586,18,22
326,2013,7,15,1618,18,1859,-39,DL,N718TW,141,JFK,SFO,312,2586,16,18


In [31]:
# Convert the local data frame into a SparkDataFrame
SFO_DF <- createDataFrame(SFO_df)

In [32]:
#  Directly create a SparkDataFrame from the source data
flightsDF <- read.df(flightsCsvPath, source = "csv", header = "true")

In [33]:
# Print the schema of this SparkDataFrame
printSchema(flightsDF)

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)


In [34]:
# Cache the SparkDataFrame
cache(flightsDF)

SparkDataFrame[year:string, month:string, day:string, dep_time:string, dep_delay:string, arr_time:string, arr_delay:string, carrier:string, tailnum:string, flight:string, origin:string, dest:string, air_time:string, distance:string, hour:string, minute:string]

In [39]:
# Print the first 6 rows of the SparkDataFrame
head(flightsDF)

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2013,6,30,940,15,1216,-4,VX,N626VA,407,JFK,LAX,313,2475,9,40
2,2013,5,7,1657,-3,2104,10,DL,N3760C,329,JFK,SJU,216,1598,16,57
3,2013,12,8,859,-1,1238,11,DL,N712TW,422,JFK,LAX,376,2475,8,59
4,2013,5,14,1841,-4,2122,-34,DL,N914DL,2391,JFK,TPA,135,1005,18,41
5,2013,7,21,1102,-3,1230,-8,9E,N823AY,3652,LGA,ORF,50,296,11,2
6,2013,1,1,1817,-3,2008,3,AA,N3AXAA,353,LGA,ORD,138,733,18,17


In [36]:
# Show the column names in the SparkDataFrame
columns(flightsDF)

[1] "year"      "month"     "day"       "dep_time"  "dep_delay" "arr_time" 
 [7] "arr_delay" "carrier"   "tailnum"   "flight"    "origin"    "dest"     
[13] "air_time"  "distance"  "hour"      "minute"

In [37]:
# Show the number of rows in the SparkDataFrame
count(flightsDF)

[1] 32735

In [45]:
# Select specific columns
destDF <- select(flightsDF, "dest", "distance")
head(destDF)

,dest,distance
,<chr>,<chr>
1,LAX,2475
2,SJU,1598
3,LAX,2475
4,TPA,1005
5,ORF,296
6,ORD,733


In [49]:
# Use collect to create a local R data frame
local_df <- collect(destDF)

In [50]:
# Print the newly created local data frame
head(local_df)

,dest,distance
,<chr>,<chr>
1,LAX,2475
2,SJU,1598
3,LAX,2475
4,TPA,1005
5,ORF,296
6,ORD,733


In [51]:
# Filter flights whose destination is JFK
jfkDF <- filter(flightsDF, "dest = \"JFK\"") ##OR
jfkDF <- filter(flightsDF, flightsDF$dest == "JFK")

In [53]:
# Stop the SparkSession now
sparkR.session.stop()